<style>
    .info-card {
        max-width: 650px;
        margin: 25px auto;
        padding: 25px 30px;
        border: 1px solid #e0e0e0;
        border-radius: 12px;
        box-shadow: 0 4px 12px rgba(0, 0, 0, 0.05);
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif;
        background-color: #fdfdfd;
        color: #333;
    }
    .info-card .title {
        color: #1a237e; /* Dark Indigo */
        font-size: 24px;
        font-weight: 600;
        margin-top: 0;
        margin-bottom: 15px;
        text-align: center;
        border-bottom: 2px solid #e8eaf6; /* Light Indigo */
        padding-bottom: 10px;
    }
    .info-card .details-grid {
        display: grid;
        grid-template-columns: max-content 1fr;
        gap: 12px 20px;
        margin-top: 20px;
        font-size: 16px;
    }
    .info-card .label {
        font-weight: 600;
        color: #555;
        text-align: right;
    }
    .info-card .value {
        font-weight: 400;
        color: #222;
    }
</style>

<div class="info-card">
    <h2 class="title">Unit 2 Exercise</h2>
    <div class="details-grid">
        <div class="label">Name:</div>
        <div class="value">Ethan Jed V. Carbonell</div>
        <div class="label">Date:</div>
        <div class="value">September 12, 2025</div>
        <div class="label">Year & Section:</div>
        <div class="value">BSCS 3A AI</div>
        <div></div>
    </div>
</div>

In [34]:
import numpy as np

class Dense_Layer:
    def __init__(self, n_inputs, n_neurons, activation_function=None):
        """
        Args:
            n_inputs (int): The number of input features (or neurons from the previous layer).
            n_neurons (int): The number of neurons in this layer.
            activation_function (str, optional): The name of the activation function to use.
                                                 Supported: 'relu', 'sigmoid', 'softmax'.
        """
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
        self.activation_function_name = activation_function
        self.inputs = None
        self.output = None
        self.activated_output = None

    def forward(self, inputs):
        """
        Args:
            inputs (np.array): Input data or outputs from the previous layer.
        
        Returns:
            np.array: The raw output of the layer (Z = X.W + b).
        """
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases
        return self.output

    def activate(self, raw_output=None):
        """
        Args:
            raw_output (np.array, optional): The raw output from the forward pass. 
                                              If None, uses the layer's stored self.output.
        
        Returns:
            np.array: The output of the layer after activation.
        """
        if raw_output is None:
            raw_output = self.output
            
        if self.activation_function_name == 'relu':
            self.activated_output = np.maximum(0, raw_output)
            
        elif self.activation_function_name == 'sigmoid':
            self.activated_output = 1 / (1 + np.exp(-raw_output))
            
        elif self.activation_function_name == 'softmax':
            exp_values = np.exp(raw_output - np.max(raw_output, axis=1, keepdims=True))
            probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
            self.activated_output = probabilities
            
        else:
            # defaults to linear activation
            self.activated_output = raw_output

        return self.activated_output

    @staticmethod
    def calculate_loss(y_pred, y_true):
        """
        Args:
            y_pred (np.array): Predicted probabilities from the output layer.
            y_true (np.array): Ground truth labels.
        
        Returns:
            float: The mean loss for the batch.
        """
        samples = len(y_pred)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        if len(y_true.shape) == 1: # Sparse
            correct_confidences = y_pred_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2: # One-hot
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        else:
            raise ValueError("Invalid shape for y_true. Must be 1D or 2D.")

        negative_log_likelihoods = -np.log(correct_confidences)
        
        data_loss = np.mean(negative_log_likelihoods)
        return data_loss

In [35]:
import numpy as np

# This is the class developed in the previous step. It is included here for completeness.
class Dense_Layer:
    """A class representing a dense (fully connected) layer in a neural network."""

    def __init__(self, n_inputs, n_neurons, weights=None, biases=None):
        """
        (1) A function to setup/accept the inputs and weights.
        """
        if weights is None or biases is None:
            self.weights = 0.10 * np.random.randn(n_inputs, n_neurons)
            self.biases = np.zeros((1, n_neurons))
        else:
            self.weights = np.array(weights)
            self.biases = np.array(biases).reshape(1, -1)
            
        if self.weights.shape != (n_inputs, n_neurons):
            raise ValueError(f"Shape of weights {self.weights.shape} does not match expected shape {(n_inputs, n_neurons)}")
        if self.biases.shape != (1, n_neurons):
            raise ValueError(f"Shape of biases {self.biases.shape} does not match expected shape {(1, n_neurons)}")

    def forward(self, inputs):
        """
        (2) A function to perform the weighted sum + bias.
        """
        self.inputs = np.array(inputs)
        self.raw_output = np.dot(self.inputs, self.weights) + self.biases
        return self.raw_output

    def activate(self, activation_function):
        """
        (3) A function to perform the selected activation function.
        """
        self.activation_name = activation_function.lower()
        
        if self.activation_name == 'relu':
            self.output = np.maximum(0, self.raw_output)
        elif self.activation_name == 'sigmoid':
            self.output = 1 / (1 + np.exp(-self.raw_output))
        elif self.activation_name == 'softmax':
            exp_values = np.exp(self.raw_output - np.max(self.raw_output, axis=1, keepdims=True))
            self.output = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        else:
            raise ValueError("Unsupported activation function. Choose from 'relu', 'sigmoid', 'softmax'.")
        
        return self.output

    @staticmethod
    def calculate_categorical_loss(y_pred, y_true):
        """
        (4) A function to calculate the loss (Categorical Cross-Entropy).
        """
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
        negative_log_likelihoods = -np.log(correct_confidences)
        return np.mean(negative_log_likelihoods)
        
    @staticmethod
    def calculate_bce_loss(y_pred, y_true):
        """
        (4) A function to calculate the loss for binary classification (Binary Cross-Entropy).
        """
        # Convert y_true to a numpy array to ensure vectorized operations work
        y_true_np = np.array(y_true)
        
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        loss = -(y_true_np * np.log(y_pred_clipped) + (1 - y_true_np) * np.log(1 - y_pred_clipped))
        return np.mean(loss)


In [40]:
if __name__ == "__main__":
    print("Iris Dataset Neural Network\n")
    inputs = [[5.1, 3.5, 1.4, 0.2]]
    
    target_output = [[0.7, 0.2, 0.1]]

    weights1 = [
        [0.2, 0.5, -0.3],
        [0.1, -0.2, 0.4],
        [-0.4, 0.3, 0.2],
        [0.6, -0.1, 0.5]
    ]
    biases1 = [3.0, -2.1, 0.6]

    weights2 = [
        [0.3, -0.5],
        [0.7, 0.2],
        [-0.6, 0.4]
    ]
    biases2 = [4.3, 6.4]

    weights3 = [
        [0.5, -0.3, 0.8],
        [-0.2, 0.6, -0.4]
    ]
    biases3 = [-1.5, 2.1, -3.3]
    
    print(f"Input data: {np.array(inputs)}")
    print(f"Target output: {np.array(target_output)}\n")

    layer1 = Dense_Layer(n_inputs=4, n_neurons=3, weights=weights1, biases=biases1)
    layer2 = Dense_Layer(n_inputs=3, n_neurons=2, weights=weights2, biases=biases2)
    layer3 = Dense_Layer(n_inputs=2, n_neurons=3, weights=weights3, biases=biases3)

    
    print("Layer 1 [ReLU]  ")
    layer1.forward(inputs)
    layer1_output = layer1.activate('relu')
    print(f"Output: {layer1_output}\n")
    
    print("Layer 2 [Sigmoid]  ")
    layer2.forward(layer1_output)
    layer2_output = layer2.activate('sigmoid')
    print(f"Output: {layer2_output}\n")

    print("Layer 3 [Softmax]  ")
    layer3.forward(layer2_output)
    final_output = layer3.activate('softmax')
    print(f"Output: {final_output}\n")

        
    print("\n----------------------------------------------------------------------------\nFINAL RESULTS\n")
    print(f"Final Predicted Output (Probabilities): {final_output}")
    
    iris_species = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
    predicted_class_index = np.argmax(final_output)
    predicted_species = iris_species[predicted_class_index]
    
    print(f"Predicted Iris Species: {predicted_species}\n")
    
    loss = Dense_Layer.calculate_categorical_loss(final_output, target_output)
    print(f"Loss (Categorical Cross-Entropy): {loss:.4f}")

Iris Dataset Neural Network

Input data: [[5.1 3.5 1.4 0.2]]
Target output: [[0.7 0.2 0.1]]

Layer 1 [ReLU]  
Output: [[3.93 0.15 0.85]]

Layer 2 [Sigmoid]  
Output: [[0.99378157 0.99187781]]

Layer 3 [Softmax]  
Output: [[0.0265075  0.96865119 0.00484132]]


----------------------------------------------------------------------------
FINAL RESULTS

Final Predicted Output (Probabilities): [[0.0265075  0.96865119 0.00484132]]
Predicted Iris Species: Iris-versicolor

Loss (Categorical Cross-Entropy): 1.5475


In [ ]:
if __name__ == "__main__":
    print("Breast Cancer Dataset Neural Network\n")

    inputs = [[14.1, 20.3, 0.095]]
    
    #1 for Malignant
    target_output = [[1]]

    weights1 = [
        [0.5, -0.3, 0.8],
        [-0.2, 0.4, -0.6],
        [-0.7, 0.9, 0.1]
    ]
    biases1 = [0.3, -0.5, 0.6]

    # Layer 2: 3 inputs, 2 neurons. Given weights are (2, 3), need to transpose to (3, 2).
    weights2_raw = [
        [0.6, -0.2, 0.4],
        [-0.3, 0.5, 0.7]
    ]
    weights2 = np.array(weights2_raw).T
    biases2 = [0.1, -0.8]

    weights3_raw = [[0.7, -0.5]]
    weights3 = np.array(weights3_raw).T
    biases3 = [0.2]
    
    print(f"Input data: {np.array(inputs)}")
    print(f"Target output: {target_output[0][0]} (Malignant)\n")

    layer1 = Dense_Layer(n_inputs=3, n_neurons=3, weights=weights1, biases=biases1)
    layer2 = Dense_Layer(n_inputs=3, n_neurons=2, weights=weights2, biases=biases2)
    layer3 = Dense_Layer(n_inputs=2, n_neurons=1, weights=weights3, biases=biases3)

    
    print("Layer 1 [ReLU]  ")
    layer1.forward(inputs)
    layer1_output = layer1.activate('relu')
    print(f"Output: {layer1_output}\n")
    
    print("Layer 2 [Sigmoid]  ")
    layer2.forward(layer1_output)
    layer2_output = layer2.activate('sigmoid')
    print(f"Output: {layer2_output}\n")

    print("Layer 3 [Sigmoid]  ")
    layer3.forward(layer2_output)
    final_output = layer3.activate('sigmoid')
    print(f"Output: {final_output}\n")
    
    
    print("\n----------------------------------------------------------------------------\nFINAL RESULTS\n")
    print(f"Final Predicted Output (Probability): {final_output[0][0]:.4f}\n")
    
    predicted_class = 1 if final_output[0][0] > 0.5 else 0
    prediction_label = "Malignant" if predicted_class == 1 else "Benign"
    
    print(f"Predicted Class: {predicted_class} ({prediction_label})\n")
    
    loss = Dense_Layer.calculate_bce_loss(final_output, target_output)
    print(f"Loss (Binary Cross-Entropy): {loss:.4f}")

Breast Cancer Dataset Neural Network

Input data: [[14.1   20.3    0.095]]
Target output: 1 (Malignant)

Layer 1 [ReLU]  
Output: [[3.2235 3.4755 0.    ]]

Layer 2 [Sigmoid]  
Output: [[0.79232544 0.49267552]]

Layer 3 [Sigmoid]  

----------------------------------------------------------------------------
FINAL RESULTS

Final Predicted Output (Probability): 0.6244

Predicted Class: 1 (Malignant)

Loss (Binary Cross-Entropy): 0.4710
